
# Flask — Autenticazione & Autorizzazione (Notebook guidato)

In questo notebook impari a:
- **Autenticarti** con sessioni Flask (login/logout, remember-me)
- Proteggere route con `@login_required`
- Implementare **Autorizzazione** con ruoli e un decorator `@roles_required(...)`
- Gestire redirect sicuri con parametro `next`
- Capire **hashing** delle password con `werkzeug.security`

> Funziona tutto **in-process**, senza avviare un server esterno: useremo il `test_client()` di Flask per simulare richieste.



## Indice
1. [Setup pacchetti](#setup)
2. [Concetti base: AuthN vs AuthZ](#concetti)
3. [App Flask minimale con ruoli](#app)
4. [Redirect sicuri (`next`)](#redirect)
5. [Test end-to-end con `test_client`](#test)
6. [Remember-me e sicurezza cookie](#remember)
7. [Hashing password: esempi](#hashing)
8. [Esercizi](#esercizi)



<a id="setup"></a>

## 1) Setup pacchetti
Esegui la cella qui sotto per installare le dipendenze nella tua environment del notebook.


In [ ]:

# Se necessario, sblocca i commenti e installa i pacchetti
# %pip install Flask Flask-Login



<a id="concetti"></a>

## 2) Concetti base: **AuthN** vs **AuthZ**
- **Autenticazione (AuthN)**: verificare *chi sei*. Esempio: login con username/password, sessione della tua identità.
- **Autorizzazione (AuthZ)**: verificare *cosa puoi fare* una volta identificato. Esempio: solo gli utenti con ruolo `admin` possono vedere `/admin`.

Nel web classico con Flask:
- L'**autenticazione** usa una **sessione** firmata (cookie) + `Flask-Login` per rendere semplice `current_user` e `@login_required`.
- L'**autorizzazione** la gestisci tu, p.es. con un campo `roles` e un decorator `@roles_required("admin")`.



<a id="app"></a>

## 3) App Flask minimale con ruoli
Qui sotto creiamo un'**Application Factory** con:
- utenti demo in memoria (hash password)
- `Flask-Login` per sessioni e `@login_required`
- decorator `roles_required` per autorizzazione a ruoli
- route `/login`, `/logout`, `/dashboard` (protetto), `/admin` (protetto e con ruolo)


In [2]:

from flask import Flask, request, redirect, url_for, abort, jsonify, session
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from werkzeug.security import generate_password_hash, check_password_hash
from urllib.parse import urlparse, urljoin
from datetime import timedelta, datetime

# -----------------------------
# Helper per redirect sicuri
# -----------------------------
def is_safe_url(target, request_host_url):
    host_url = urlparse(request_host_url)
    test_url = urlparse(urljoin(request_host_url, target))
    return (test_url.scheme in ("http", "https") and host_url.netloc == test_url.netloc)

# -----------------------------
# Decorator di autorizzazione
# -----------------------------
def roles_required(*required_roles):
    def wrapper(fn):
        from functools import wraps
        @wraps(fn)
        def inner(*args, **kwargs):
            if not current_user.is_authenticated:
                abort(401)
            user_roles = getattr(current_user, "roles", []) or []
            if not any(r in user_roles for r in required_roles):
                abort(403)
            return fn(*args, **kwargs)
        return inner
    return wrapper

# -----------------------------
# User model "light" + storage in-memory
# -----------------------------
class User(UserMixin):
    def __init__(self, id, username, password_hash, roles):
        self.id = str(id)
        self.username = username
        self.password_hash = password_hash
        self.roles = roles

    def check_password(self, password):
        return check_password_hash(self.password_hash, password)

# Creiamo un piccolo "DB" in memoria
def seed_users():
    return {
        "giulia": User(1, "giulia", generate_password_hash("ciao123"), roles=["admin"]),
        "luca":   User(2, "luca",   generate_password_hash("guest123"), roles=["guest"]),
    }

def create_app():
    app = Flask(__name__)
    app.config.update(
        SECRET_KEY="dev-secret-change-me",  # in prod: chiave lunga e segreta
        REMEMBER_COOKIE_DURATION=timedelta(days=7),
        SESSION_COOKIE_SAMESITE="Lax",
        SESSION_COOKIE_SECURE=False,  # True se usi HTTPS
        PERMANENT_SESSION_LIFETIME=timedelta(hours=8),
    )

    login_manager = LoginManager(app)
    login_manager.login_view = "login"  # per @login_required

    USERS = seed_users()
    ID_INDEX = {u.id: u for u in USERS.values()}

    @login_manager.user_loader
    def load_user(user_id):
        return ID_INDEX.get(str(user_id))

    @app.route("/")
    def home():
        if current_user.is_authenticated:
            return jsonify(message=f"Ciao {current_user.username}!", roles=current_user.roles)
        return jsonify(message="Ciao visitatore! Fai login su /login (POST).")

    @app.route("/login", methods=["POST"])
    def login():
        data = request.get_json(silent=True) or request.form or {}
        username = data.get("username")
        password = data.get("password")
        remember = bool(data.get("remember", False))
        next_url = data.get("next") or request.args.get("next")

        user = USERS.get(username)
        if not user or not user.check_password(password):
            abort(401)  # credenziali errate

        login_user(user, remember=remember)

        # Redirect sicuro
        if next_url and is_safe_url(next_url, request.host_url):
            return redirect(next_url, code=303)

        return jsonify(ok=True, user=user.username, roles=user.roles)

    @app.route("/logout", methods=["POST"])
    @login_required
    def logout():
        logout_user()
        return jsonify(ok=True)

    @app.route("/dashboard")
    @login_required
    def dashboard():
        return jsonify(message=f"Benvenuto nella dashboard, {current_user.username}!")

    @app.route("/admin")
    @login_required
    @roles_required("admin")
    def admin():
        return jsonify(message=f"Area admin: accesso consentito a {current_user.username}.")

    @app.route("/api/me")
    @login_required
    def me():
        return jsonify(username=current_user.username, roles=current_user.roles)

    return app

app = create_app()
app  # mostriamo solo che esiste


<Flask '__main__'>


<a id="redirect"></a>

## 4) Redirect sicuri con `next`
Quando fai login dopo un 302 da una pagina protetta, spesso vuoi tornare alla pagina di origine (`next`).
**Mai** fidarsi ciecamente di `next`: valida il dominio con una funzione tipo `is_safe_url(...)` per evitare **open redirect**.
Nel codice sopra, se `next` non è valido, rispondiamo con JSON standard.



<a id="test"></a>

## 5) Test end-to-end con `test_client`
Simuliamo richieste come farebbe un browser. Vediamo:
1. Accesso a `/dashboard` senza login → **302** verso `login_view` o **401/302** a seconda della config
2. Login **ok** come `giulia` (admin)
3. Accesso a `/admin` (consentito)
4. Logout
5. Login come `luca` (guest) e tentativo su `/admin` → **403**


In [ ]:

from urllib.parse import urlparse

def print_response(r, label):
    print(f"[{label}] status={r.status_code}")
    ct = r.headers.get("Content-Type", "")
    if "application/json" in ct:
        print(" body:", r.get_json())
    else:
        loc = r.headers.get("Location")
        if loc:
            print(" redirect to:", loc)

with app.test_client() as client:
    # 1) /dashboard senza login
    r1 = client.get("/dashboard", follow_redirects=False)
    print_response(r1, "1 dashboard senza login")

    # 2) login giulia (admin)
    r2 = client.post("/login", json={"username":"giulia", "password":"ciao123", "remember": True})
    print_response(r2, "2 login giulia")

    # 3) admin ok
    r3 = client.get("/admin")
    print_response(r3, "3 admin con giulia")

    # 4) logout
    r4 = client.post("/logout")
    print_response(r4, "4 logout")

    # 5) login luca (guest) + admin -> 403
    r5 = client.post("/login", json={"username":"luca", "password":"guest123"})
    print_response(r5, "5 login luca")
    r6 = client.get("/admin")
    print_response(r6, "6 admin con luca (atteso 403)")



<a id="remember"></a>

## 6) Remember-me e sicurezza cookie
- `login_user(user, remember=True)` imposta un cookie **remember** che mantiene l'accesso oltre la sessione.
- Config utili (già nel nostro `create_app()`):
  - `REMEMBER_COOKIE_DURATION=timedelta(days=7)`
  - `SESSION_COOKIE_SAMESITE="Lax"`
  - `SESSION_COOKIE_SECURE=True` **in produzione con HTTPS**
  - `SECRET_KEY` **lungo e segreto**

### Pattern PRG (Post/Redirect/Get)
Dopo un `POST /login`, manda spesso un **303** verso una pagina **GET** (dashboard). Nel nostro esempio, se `next` è valido, facciamo `redirect(..., code=303)`.



<a id="hashing"></a>

## 7) Hashing password: esempi
Usiamo `werkzeug.security`:
- `generate_password_hash("pwd")` → stringa con algoritmo e salt (es. `pbkdf2:sha256:260000$...`)
- `check_password_hash(hash, "pwd")` → `True/False`

> **Mai** salvare password in chiaro. Valuta anche policy su lunghezza minima, rate limiting ai login, e CAPTCHA dopo troppi tentativi.


In [ ]:

from werkzeug.security import generate_password_hash, check_password_hash

demo_hash = generate_password_hash("esempio123")
print("Hash generato:", demo_hash)
print("Verifica ok:", check_password_hash(demo_hash, "esempio123"))
print("Verifica KO:", check_password_hash(demo_hash, "sbagliata"))



<a id="esercizi"></a>

## 8) Esercizi (per te)
1. **Aggiungi un ruolo** `editor` e crea una route `/editor` permessa a `admin` **e** `editor`.
2. Implementa un campo `remember` nella versione **form** (non JSON) di `/login` e prova il cookie di remember.
3. Crea un endpoint `POST /change-password` (protetto) che accetta `old_password` e `new_password` e aggiorna l'hash (in questo notebook, modifica l'utente in memoria).
4. Aggiungi rate limiting (es. con `Flask-Limiter`) per limitare tentativi sbagliati di login.
5. Bonus: sposta gli utenti in un **database** con SQLAlchemy e crea una migrazione Alembic.

---

### Riferimenti utili
- Documentazione: Flask-Login
- Sicurezza cookie: Flask docs (Session, Security)
- Werkzeug security utils
